In [43]:
import requests
import random
import yt_dlp
import pandas as pd
import threading
import time
from concurrent.futures import ThreadPoolExecutor
import socket


# URLs để tải proxy và file CSV
PROXY_FILES = {
    "http": "https://raw.githubusercontent.com/r00tee/Proxy-List/main/Https.txt",
    "socks4": "https://raw.githubusercontent.com/r00tee/Proxy-List/main/Socks4.txt",
    "socks5": "https://raw.githubusercontent.com/r00tee/Proxy-List/main/Socks5.txt"
}

CSV_URL = "https://huggingface.co/datasets/Loie/VGGSound/resolve/main/vggsound.csv"

# Cấu hình yt-dlp
ydl_opts = {
    'outtmpl': './videos/%(id)s.%(ext)s',
    'format': 'bestvideo+bestaudio/best',
    'socket_timeout': 60,  # Tăng timeout lên 60 giây
    'http_chunk_size': 1048576,  # Giảm tải từng phần nhỏ
}

In [44]:
# Tải file CSV
def load_video_data(csv_url):
    df = pd.read_csv(csv_url)
    df.columns = ['YouTube ID', 'start_seconds', 'label', 'split']
    return df['YouTube ID'].tolist()

# Tải danh sách proxy từ URL
def get_proxy_list(proxy_url):
    try:
        response = requests.get(proxy_url)
        response.raise_for_status()
        proxies = response.text.splitlines()
        proxies = [proxy.strip() for proxy in proxies if proxy.strip()]
        return proxies
    except Exception as e:
        print(f"Lỗi khi tải proxy từ {proxy_url}: {e}")
        return []

# Kiểm tra proxy
def check_proxy(proxy):
    try:
        host, port = proxy.split(':')
        socket.create_connection((host, int(port)), timeout=5)
        return True
    except:
        return False

# Tải video từ YouTube với proxy
def download_video_with_proxy(video_id, proxies):
    url = f"https://www.youtube.com/watch?v={video_id}"
    while True:
        try:
            # Chọn proxy ngẫu nhiên
            proxy = random.choice(proxies)
            print(f"Đang thử proxy: {proxy}")
            ydl_opts['proxy'] = proxy
            # Tải video
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                ydl.download([url])
            print(f"Đã tải thành công video: {video_id}")
            break
        except Exception as e:
            print(f"Lỗi khi tải video {video_id} với proxy {proxy}: {e}")
            time.sleep(5)  # Chờ trước khi thử lại

# Cập nhật proxy tự động mỗi 5 phút
def update_proxies(proxy_type, proxy_list):
    while True:
        print(f"Đang cập nhật proxy {proxy_type}...")
        new_proxies = get_proxy_list(PROXY_FILES[proxy_type])
        if new_proxies:
            proxy_list.clear()
            proxy_list.extend(new_proxies)
            print(f"Đã cập nhật {len(new_proxies)} proxy {proxy_type}.")
        else:
            print(f"Không thể tải proxy {proxy_type}.")
        time.sleep(300)  # Cập nhật mỗi 5 phút

In [ ]:
# Tải video đa luồng
def main():
    proxy_type = "http"  # Chọn loại proxy (http, socks4, socks5)
    proxy_list = get_proxy_list(PROXY_FILES[proxy_type])

    if not proxy_list:
        print("Không thể tải danh sách proxy. Dừng chương trình.")
        return

    # Tải danh sách video từ CSV
    videos = load_video_data(CSV_URL)

    # Luồng cập nhật proxy
    threading.Thread(target=update_proxies, args=(proxy_type, proxy_list), daemon=True).start()

    # Tải video đa luồng
    with ThreadPoolExecutor(max_workers=5) as executor:  # Tăng max_workers để tải nhiều video hơn
        for video_id in videos[:10]:  # Tải 10 video đầu tiên
            executor.submit(download_video_with_proxy, video_id, proxy_list)

if __name__ == "__main__":
    main()

Đang thử proxy: 196.20.125.121:8083Lỗi khi tải video --96EN9NUQM với proxy 196.20.125.121:8083: Socket operation on non-socket
Đang thử proxy: 210.201.86.72:8080Lỗi khi tải video --96EN9NUQM với proxy 210.201.86.72:8080: Socket operation on non-socket
Đang thử proxy: 198.55.103.153:5672Lỗi khi tải video --96EN9NUQM với proxy 198.55.103.153:5672: Socket operation on non-socket
Đang thử proxy: 110.171.40.132:8080Lỗi khi tải video --96EN9NUQM với proxy 110.171.40.132:8080: Socket operation on non-socket
Đang thử proxy: 138.68.235.51:80Lỗi khi tải video --96EN9NUQM với proxy 138.68.235.51:80: Socket operation on non-socket


[youtube] --56QUhyDQM: Downloading mweb player API JSON
Đang thử proxy: 8.215.15.163:8008Lỗi khi tải video --96EN9NUQM với proxy 8.215.15.163:8008: Socket operation on non-socket


[youtube] --0PQM4-hqg: Downloading ios player API JSON


Đang thử proxy: 185.170.233.103:47574Lỗi khi tải video --96EN9NUQM với proxy 185.170.233.103:47574: Socket operation on non-socket
[youtube] --56QUhyDQM: Downloading initial data API JSON


ERROR: [youtube] --56QUhyDQM: Requested format is not available. Use --list-formats for a list of available formats


Lỗi khi tải video --56QUhyDQM với proxy 160.20.165.230:8587: ERROR: [youtube] --56QUhyDQM: Requested format is not available. Use --list-formats for a list of available formats
Đang thử proxy: 67.201.58.190:4145Lỗi khi tải video --96EN9NUQM với proxy 67.201.58.190:4145: Socket operation on non-socket
Đang thử proxy: 103.247.14.103:1111
[youtube] Extracting URL: https://www.youtube.com/watch?v=--56QUhyDQM
[youtube] --56QUhyDQM: Downloading webpage
Đang thử proxy: 103.189.249.210:8080Lỗi khi tải video --96EN9NUQM với proxy 103.189.249.210:8080: Socket operation on non-socket


[youtube] --8puiAGLhs: Downloading mweb player API JSON
Đang thử proxy: 13.80.134.180:80Lỗi khi tải video --96EN9NUQM với proxy 13.80.134.180:80: Socket operation on non-socket
Đang thử proxy: 80.26.56.66:4153
[youtube] Extracting URL: https://www.youtube.com/watch?v=--5OkAjCI7g
[youtube] --5OkAjCI7g: Downloading webpage
Đang thử proxy: 121.101.133.81:1111Lỗi khi tải video --96EN9NUQM với proxy 121.101.133.81:1111: Socket operation on non-socket
Đang thử proxy: 20.120.225.109:3128Lỗi khi tải video --96EN9NUQM với proxy 20.120.225.109:3128: Socket operation on non-socket


[youtube] --56QUhyDQM: Downloading ios player API JSON
Đang thử proxy: 103.106.231.188:42020Lỗi khi tải video --96EN9NUQM với proxy 103.106.231.188:42020: Socket operation on non-socket


In [46]:
import yt_dlp

# URL video cần tải
url = "https://www.youtube.com/watch?v=--96EN9NUQM"

# Proxy đã chọn
selected_proxy = "93.157.196.58:8080"  # Thay bằng proxy đã chọn từ bước trên

# Cấu hình yt-dlp
ydl_opts = {
    'outtmpl': './videos/%(id)s.%(ext)s',
    'format': 'bestvideo+bestaudio/best',
    'proxy': selected_proxy,  # Thêm proxy vào đây
}

# Tải video
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([url])
print("Tải video thành công!")


[youtube] Extracting URL: https://www.youtube.com/watch?v=--96EN9NUQM
[youtube] --96EN9NUQM: Downloading webpage


[youtube] --96EN9NUQM: Downloading ios player API JSON


[youtube] --96EN9NUQM: Downloading ios player API JSON


[youtube] --96EN9NUQM: Downloading ios player API JSON


[youtube] --96EN9NUQM: Downloading ios player API JSON


...

[youtube] --96EN9NUQM: Downloading mweb player API JSON


ERROR: [youtube] --5OkAjCI7g: Failed to extract any player response; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U


Lỗi khi tải video --5OkAjCI7g với proxy 220.167.233.129:8118: ERROR: [youtube] --5OkAjCI7g: Failed to extract any player response; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U


ERROR: [youtube] --96EN9NUQM: Failed to extract any player response; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U


DownloadError: ERROR: [youtube] --96EN9NUQM: Failed to extract any player response; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U